In [ ]:
pip install psutil

In [ ]:
from pulp import *
import numpy as np
import random
from dataclasses import dataclass
import multiprocessing
import psutil
import time
from typing import List, Dict, Tuple, Set, Optional, Union
from pulp import PULP_CBC_CMD
import signal
from multiprocessing import Process

@dataclass
class TBRDInstance:
    """Class to hold a TBRD problem instance"""
    C: Set[str]  # Customers
    D: Set[str]  # Drop-off points
    R: Set[str]  # Robot depots
    deadlines: Dict[str, float]  # Customer deadlines
    weights: Dict[str, float]    # Customer weights
    initial_robots: int          # Number of robots initially loaded (δ)
    truck_capacity: int          # Maximum loading capacity for robots (K)
    truck_travel_times: Dict[Tuple[str, str], float]  # Travel times for truck
    robot_travel_times: Dict[Tuple[str, str], float]  # Travel times for robots
    coordinates: Dict[str, Tuple[float, float]]       # Coordinates of all points

class TBRDInstanceGenerator:
    def __init__(self, 
                 w: float,                # Square side length (km)
                 num_customers: int,      # Number of customers
                 num_dropoff: int,        # Number of drop-off points
                 num_depots: int,         # Number of robot depots
                 truck_speed: float,      # Truck speed (km/h)
                 robot_speed: float,      # Robot speed (km/h)
                 truck_capacity: int,     # Truck capacity (K)
                 rho_min: float,          # Minimum deadline factor
                 rho_max: float,          # Maximum deadline factor
                 w_min: float,            # Minimum customer weight
                 w_max: float):           # Maximum customer weight
        self.w = w
        self.num_customers = num_customers
        self.num_dropoff = num_dropoff
        self.num_depots = num_depots
        self.truck_speed = truck_speed
        self.robot_speed = robot_speed
        self.truck_capacity = truck_capacity
        self.rho_min = rho_min
        self.rho_max = rho_max
        self.w_min = w_min
        self.w_max = w_max
        self.grid_size = 1/6  # As specified in the paper

    def generate_instance(self, seed: Optional[int] = None) -> TBRDInstance:
        """Generate a single TBRD instance"""
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)
            
        # Generate coordinates for all points
        coordinates = self._generate_layout()
        
        # Calculate travel times
        truck_travel_times = self._calculate_truck_travel_times(coordinates)
        robot_travel_times = self._calculate_robot_travel_times(coordinates)
        
        # Generate customer deadlines
        deadlines = self._generate_deadlines(coordinates, truck_travel_times, robot_travel_times)
        
        # Generate customer weights
        weights = self._generate_weights()
        
        # Create customer, drop-off, and depot sets
        C = {f'C{i}' for i in range(self.num_customers)}
        D = {f'D{i}' for i in range(self.num_dropoff)}
        R = {f'R{i}' for i in range(self.num_depots)}
        
        return TBRDInstance(
            C=C,
            D=D,
            R=R,
            deadlines=deadlines,
            weights=weights,
            initial_robots=self.truck_capacity,  # δ = K as specified
            truck_capacity=self.truck_capacity,
            truck_travel_times=truck_travel_times,
            robot_travel_times=robot_travel_times,
            coordinates=coordinates
        )

    def _generate_layout(self) -> Dict[str, Tuple[float, float]]:
        """Generate geographical layout of depots, drop-off points, and customers"""
        coordinates = {}
        
        # Generate equidistant robot depots
        depots_per_side = int(np.ceil(np.sqrt(self.num_depots)))
        spacing = self.w / (depots_per_side + 1)
        
        depot_idx = 0
        for i in range(1, depots_per_side + 1):
            for j in range(1, depots_per_side + 1):
                if depot_idx < self.num_depots:
                    coordinates[f'R{depot_idx}'] = (i * spacing, j * spacing)
                    depot_idx += 1
        
        # Generate random drop-off points
        used_positions = set((x, y) for x, y in coordinates.values())
        for i in range(self.num_dropoff):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'D{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Generate random customer positions
        for i in range(self.num_customers):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'C{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Set initial truck position (γ)
        possible_starts = [k for k in coordinates.keys() if k.startswith(('R', 'D'))]
        gamma = random.choice(possible_starts)
        coordinates['gamma'] = coordinates[gamma]
        
        return coordinates

    def _calculate_euclidean_distance(self, 
                                    p1: Tuple[float, float], 
                                    p2: Tuple[float, float]) -> float:
        """Calculate Euclidean distance between two points"""
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def _calculate_truck_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, str], float]:
        """Calculate truck travel times between all relevant points"""
        travel_times = {}
        locations = [k for k in coordinates.keys() if k != 'gamma_e']
        
        for i in locations:
            for j in locations:
                if i != j:
                    distance = self._calculate_euclidean_distance(
                        coordinates[i], coordinates[j])
                    travel_times[(i, j)] = distance / self.truck_speed
        
        return travel_times

    def _calculate_robot_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, str], float]:
        """Calculate robot travel times from all possible launch points to customers"""
        travel_times = {}
        launch_points = [k for k in coordinates.keys() if k.startswith(('R', 'D', 'gamma'))]
        customers = [k for k in coordinates.keys() if k.startswith('C')]
        
        for i in launch_points:
            for k in customers:
                distance = self._calculate_euclidean_distance(
                    coordinates[i], coordinates[k])
                travel_times[(i, k)] = distance / self.robot_speed
        
        return travel_times

    def _generate_deadlines(self, 
                          coordinates: Dict[str, Tuple[float, float]],
                          truck_times: Dict[Tuple[str, str], float],
                          robot_times: Dict[Tuple[str, str], float]
                          ) -> Dict[str, float]:
        """Generate customer deadlines based on minimum travel times"""
        deadlines = {}
        customers = [k for k in coordinates.keys() if k.startswith('C')]
        
        for k in customers:
            # Calculate minimum travel time to customer k
            min_time = float('inf')
            for j in [p for p in coordinates.keys() if p.startswith(('R', 'D'))]:
                if ('gamma', j) in truck_times and (j, k) in robot_times:
                    total_time = truck_times[('gamma', j)] + robot_times[(j, k)]
                    min_time = min(min_time, total_time)
            
            # Generate random factor and calculate deadline
            r_k = random.uniform(0, 1)
            deadlines[k] = min_time * (self.rho_min + (self.rho_max - self.rho_min) * r_k)
            
        return deadlines

    def _generate_weights(self) -> Dict[str, float]:
        """Generate random customer weights"""
        return {f'C{k}': random.uniform(self.w_min, self.w_max) 
                for k in range(self.num_customers)}

class TBRDModel:
    def __init__(self, instance: TBRDInstance):
        """Initialize TBRD model with problem instance"""
        self.instance = instance
        self.gamma = 'gamma'
        self.gamma_e = 'gamma_e'
        
        # Calculate big M based on problem parameters
        self.M = self._calculate_big_M()
        
        # Create PuLP model
        self.model = LpProblem("TBRD", LpMinimize)
        
        self._create_variables()
        self._add_constraints()
        self._set_objective()

    def solve(self):
        # Set a 300-second timeout for the solver
        self.model.solve(PULP_CBC_CMD(timeLimit=300))
    
    def _calculate_big_M(self) -> float:
        """Calculate appropriate Big M value"""
        max_travel_time = max(self.instance.truck_travel_times.values(), default=0)
        max_deadline = max(self.instance.deadlines.values(), default=0)
        max_robot_time = max(self.instance.robot_travel_times.values(), default=0)
        return 2 * (max_travel_time + max_deadline + max_robot_time)

    def _create_variables(self):
        """Create the decision variables for the model"""
        # l[j]: Amount of robots on board at departure from location j
        self.l = LpVariable.dicts("load",
            self.instance.D.union({self.gamma}),
            lowBound=0
        )
        
        # s[i,j]: 1 if location j is direct successor of i
        locations = self.instance.D.union(self.instance.R).union({self.gamma})
        locations_plus = self.instance.D.union(self.instance.R).union({self.gamma_e})
        self.s = LpVariable.dicts("successor",
            [(i, j) for i in locations for j in locations_plus if i != j],
            cat='Binary'
        )
        
        # t[j]: Arrival time at location j
        self.t = LpVariable.dicts("time",
            locations,
            lowBound=0
        )
        
        # x[j,k]: 1 if customer k is supplied from location j
        self.x = LpVariable.dicts("supply",
            [(j, k) for j in locations for k in self.instance.C],
            cat='Binary'
        )
        
        # z[k]: 1 if customer k is supplied late
        self.z = LpVariable.dicts("late",
            self.instance.C,
            cat='Binary'
        )

    def _add_constraints(self):
        """Add all constraints to the model"""
        self._add_customer_mapping_constraints()
        self._add_loading_constraints()
        self._add_timing_constraints()
        self._add_routing_constraints()

    def _add_customer_mapping_constraints(self):
        """Add constraints for customer mapping"""
        locations = self.instance.D.union(self.instance.R).union({self.gamma})
        
        # Constraint (2): One-to-one mapping
        for k in self.instance.C:
            self.model += lpSum(self.x[j,k] for j in locations) == 1

    def _add_loading_constraints(self):
        """Add constraints for robot loading"""
        # Constraint (3): Initial loading
        self.model += (
            self.l[self.gamma] <= self.instance.initial_robots - 
            lpSum(self.x[self.gamma,k] for k in self.instance.C)
        )
        
        # Constraints (4-5): Loading after locations
        for i in self.instance.R:
            for j in self.instance.D:
                self.model += (
                    self.l[j] <= self.instance.truck_capacity + 
                    self.M * (1 - self.s[i,j]) -
                    lpSum(self.x[j,k] for k in self.instance.C)
                )
        
        for i in self.instance.D.union({self.gamma}):
            for j in self.instance.D:
                if i != j:
                    self.model += (
                        self.l[j] <= self.l[i] + 
                        self.M * (1 - self.s[i,j]) -
                        lpSum(self.x[j,k] for k in self.instance.C)
                    )

    def _add_timing_constraints(self):
        """Add constraints for timing"""
        # Constraint (6): Initial time
        self.model += self.t[self.gamma] == 0
        
        # Constraint (7): Arrival times
        for i in self.instance.D.union(self.instance.R).union({self.gamma}):
            for j in self.instance.D.union(self.instance.R):
                if i != j:
                    self.model += (
                        self.t[j] >= self.t[i] - 
                        self.M * (1 - self.s[i,j]) +
                        self.instance.truck_travel_times.get((i,j), 0)
                    )
        
        # Constraint (8): Late delivery determination
        for j in self.instance.D.union(self.instance.R).union({self.gamma}):
            for k in self.instance.C:
                self.model += (
                    self.M * self.z[k] >= 
                    self.t[j] - self.M * (1 - self.x[j,k]) +
                    self.instance.robot_travel_times.get((j,k), 0) - 
                    self.instance.deadlines[k]
                )

    def _add_routing_constraints(self):
        """Add constraints for routing"""
        # Constraint (9): Initial location
        self.model += (
            lpSum(self.s[self.gamma,j] for j in 
                 self.instance.D.union(self.instance.R).union({self.gamma_e})) <= 1
        )
        
        # Constraint (10): Flow conservation
        for j in self.instance.D.union(self.instance.R):
            self.model += (
                lpSum(self.s[j,i] for i in 
                     self.instance.D.union(self.instance.R).union({self.gamma_e}) - {j}) ==
                lpSum(self.s[i,j] for i in 
                     self.instance.D.union(self.instance.R).union({self.gamma}) - {j})
            )
        
        # Constraints (11-13): Valid tour constraints
        for j in self.instance.D.union(self.instance.R):
            # Constraint (11): Launch from visited locations only
            self.model += (
                lpSum(self.x[j,k] for k in self.instance.C) <=
                self.M * lpSum(self.s[i,j] for i in 
                             self.instance.D.union(self.instance.R).union({self.gamma}) - {j})
            )
        
        # Constraint (12): Drop-off points usage
        for j in self.instance.D:
            self.model += (
                lpSum(self.x[j,k] for k in self.instance.C) >=
                lpSum(self.s[i,j] for i in 
                     self.instance.D.union(self.instance.R).union({self.gamma}) - {j})
            )
        
        # Constraint (13): Robot depot usage
        for j in self.instance.R:
            self.model += (
                lpSum(self.x[j,k] for k in self.instance.C) + 
                lpSum(self.s[j,i] for i in self.instance.D) >=
                lpSum(self.s[i,j] for i in 
                     self.instance.D.union(self.instance.R).union({self.gamma}) - {j})
            )

    def _set_objective(self):
        """Set the objective function (1)"""
        self.model += lpSum(self.z[k] * self.instance.weights[k] for k in self.instance.C)

    def solve(self, time_limit: Optional[int] = None, gap: Optional[float] = None) -> int:
        """
        Solve the model with optional time limit and gap
        """
        try:
            # Create solver with CBC options as a string
            solver = PULP_CBC_CMD(
                path=None,  # Use default CBC
                msg=True,  # Show solver output
                options=['timeMode elapsed',
                        f'sec {time_limit}' if time_limit else '',
                        f'ratio {gap}' if gap else '',
                        'strong 10',
                        'feasibilityPump on',
                        'heuristics on']
            )
            
            print("\nProblem Statistics:")
            print(f"Number of variables: {len(self.model.variables())}")
            print(f"Number of constraints: {len(self.model.constraints)}")
        
            # Start solving
            start_time = time.time()
            status = self.model.solve(solver)
            solve_time = time.time() - start_time
        
            if time_limit and solve_time >= time_limit:
                print(f"\nWARNING: Time limit of {time_limit} seconds was reached")
                return 0
            
            print(f"\nSolution Status: {LpStatus[status]}")
            return status
        
        except Exception as e:
            print(f"\nERROR: Solver failed with error: {str(e)}")
            return 0
        
        finally:
            # Clean up
            try:
                subprocess.run(['taskkill', '/F', '/IM', 'cbc.exe'], 
                             stdout=subprocess.DEVNULL, 
                             stderr=subprocess.DEVNULL)
                subprocess.run(['del', 'solve.bat'], shell=True)
            except:
                pass

    def get_solution(self) -> Optional[Dict]:
        """
        Return the solution details if any feasible solution was found
    
        Returns:
        --------
        dict or None
            Solution details if any solution found, None otherwise
        """
        # Accept both optimal and feasible solutions
        if self.model.status in [LpStatusOptimal, LpStatusNotSolved]:
            try:
                # Check if we have any solution values
                if not any(var.varValue is not None for var in self.model.variables()):
                    print("\nNo feasible solution found")
                    return None
                
                solution = {
                    'objective_value': value(self.model.objective),
                    'late_deliveries': {k: value(self.z[k]) for k in self.instance.C},
                    'route': self._extract_route(),
                    'customer_assignments': self._extract_assignments(),
                    'coordinates': self.instance.coordinates,
                    'completion_time': self.model.solutionTime if hasattr(self.model, 'solutionTime') else None,
                    'is_optimal': self.model.status == LpStatusOptimal
                }
            
                print("\nDetailed Solution:")
                print(f"Solution Status: {'Optimal' if solution['is_optimal'] else 'Feasible'}")
                print(f"Objective Value: {solution['objective_value']}")
                print("Route:", ' -> '.join(solution['route']))
                print("Customer Assignments:")
                for customer, location in solution['customer_assignments'].items():
                    print(f"  {customer} served from {location}")
                print("Late Deliveries:", {k: v for k, v in solution['late_deliveries'].items() if v > 0.001})
            
                return solution
            except Exception as e:
                print(f"\nError extracting solution: {str(e)}")
                return None
        else:
            print(f"\nNo solution available. Status: {LpStatus[self.model.status]}")
            return None

    def _extract_route(self) -> List[str]:
        """Extract the truck route from the solution"""
        route = []
        current = self.gamma
        while current != self.gamma_e:
            route.append(current)
            for j in self.instance.D.union(self.instance.R).union({self.gamma_e}):
                if j != current and value(self.s[current,j]) > 0.5:
                    current = j
                    break
        return route

    def _extract_assignments(self) -> Dict[str, str]:
        """Extract customer-to-location assignments from the solution"""
        assignments = {}
        for k in self.instance.C:
            for j in self.instance.D.union(self.instance.R).union({self.gamma}):
                if value(self.x[j,k]) > 0.5:
                    assignments[k] = j
        return assignments


def run_experiment(instance_params: Dict, 
                  num_instances: int = 1,
                  time_limit: Optional[int] = None,
                  gap: Optional[float] = None) -> List[Dict]:
    """
    Run experiments with specified parameters
    """
    results = []
    best_objective = float('inf')
    best_solutions_count = 0
    best_solutions_total_time = 0
    
    # Create instance generator
    generator = TBRDInstanceGenerator(**instance_params)
    
    for i in range(num_instances):
        print(f"\nSolving instance {i+1}/{num_instances}")
        
        # Generate instance with seed for reproducibility
        instance = generator.generate_instance(seed=i)
        
        print("\nInstance Details:")
        print(f"Customers: {len(instance.C)}")
        print(f"Drop-off points: {len(instance.D)}")
        print(f"Robot depots: {len(instance.R)}")
        
        # Create and solve model
        model = TBRDModel(instance)
        status = model.solve(time_limit=time_limit, gap=gap)
        
        # Get solution
        solution = model.get_solution()
        if solution:
            solution['instance_id'] = i
            solution['status'] = status
            results.append(solution)
            
            # Update best solution tracking
            if solution['objective_value'] < best_objective:
                best_objective = solution['objective_value']
                best_solutions_count = 1
                best_solutions_total_time = solution['completion_time']
            elif solution['objective_value'] == best_objective:
                best_solutions_count += 1
                best_solutions_total_time += solution['completion_time']
        else:
            print(f"\nNo solution found for instance {i+1}")
    
    # Print only final summary statistics
    if results:
        print("\n" + "="*50)
        print("FINAL SUMMARY STATISTICS")
        print("="*50)
        print(f"Total instances solved: {len(results)}")
        print(f"\nBest Solution Statistics:")
        print(f"Best objective value found: {best_objective}")
        print(f"Number of times best solution was found: {best_solutions_count}")
        if best_solutions_count > 0:
            print(f"Average CPU time for best solutions: {best_solutions_total_time/best_solutions_count:.2f} seconds")
        
        print(f"\nOverall Statistics:")
        print(f"Average objective value: {sum(r['objective_value'] for r in results) / len(results):.2f}")
        print(f"Average solution time: {sum(r['completion_time'] for r in results) / len(results):.2f} seconds")
        print(f"Average number of late deliveries: {sum(sum(1 for v in r['late_deliveries'].values() if v > 0.5) for r in results) / len(results):.2f}")
    else:
        print("\nNo solutions found.")
    
    return results

if __name__ == "__main__":
    # Medium-sized instance with more relaxed parameters
    instance_params = {
        'w': 5,                # 5 km square area (increased from 2)
        'num_customers': 10,   # 10 customers
        'num_dropoff': 5,      # 5 drop-off points
        'num_depots': 3,       # 3 robot depots
        'truck_speed': 40,     # 40 km/h (increased from 30)
        'robot_speed': 20,     # 20 km/h (increased from 15)
        'truck_capacity': 5,   # Maximum 5 robots (increased from 3)
        'rho_min': 1.5,        # Minimum deadline factor (increased from 1.2)
        'rho_max': 3.0,        # Maximum deadline factor (increased from 2.0)
        'w_min': 1,           # Minimum customer weight
        'w_max': 1            # Maximum customer weight
    }
    
    # Run experiment with two instances
    results = run_experiment(
        instance_params,
        num_instances=10,      # Two instances
        time_limit=300,       # 5 minutes per instance
        gap=0.1              # 10% gap tolerance
    )

In [11]:
from pulp import *
import numpy as np
import random
from dataclasses import dataclass
import multiprocessing
import psutil
import time
from typing import List, Dict, Tuple, Set, Optional, Union
from pulp import PULP_CBC_CMD
import signal
from multiprocessing import Process

@dataclass
class TBRDInstance:
    """Class to hold a TBRD problem instance"""
    C: Set[str]  # Customers
    D: Set[str]  # Drop-off points
    R: Set[str]  # Robot depots
    deadlines: Dict[str, float]  # Customer deadlines
    weights: Dict[str, float]    # Customer weights
    initial_robots: int          # Number of robots initially loaded (δ)
    truck_capacity: int          # Maximum loading capacity for robots (K)
    truck_travel_times: Dict[Tuple[str, str], float]  # Travel times for truck
    robot_travel_times: Dict[Tuple[str, str], float]  # Travel times for robots
    coordinates: Dict[str, Tuple[float, float]]       # Coordinates of all points

class TBRDInstanceGenerator:
    def __init__(self, 
                 w: float,                # Square side length (km)
                 num_customers: int,      # Number of customers
                 num_dropoff: int,        # Number of drop-off points
                 num_depots: int,         # Number of robot depots
                 truck_speed: float,      # Truck speed (km/h)
                 robot_speed: float,      # Robot speed (km/h)
                 truck_capacity: int,     # Truck capacity (K)
                 rho_min: float,          # Minimum deadline factor
                 rho_max: float,          # Maximum deadline factor
                 w_min: float,            # Minimum customer weight
                 w_max: float):           # Maximum customer weight
        self.w = w
        self.num_customers = num_customers
        self.num_dropoff = num_dropoff
        self.num_depots = num_depots
        self.truck_speed = truck_speed
        self.robot_speed = robot_speed
        self.truck_capacity = truck_capacity
        self.rho_min = rho_min
        self.rho_max = rho_max
        self.w_min = w_min
        self.w_max = w_max
        self.grid_size = 1/6  # As specified in the paper

    def generate_instance(self, seed: Optional[int] = None) -> TBRDInstance:
        """Generate a single TBRD instance"""
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)
            
        # Generate coordinates for all points
        coordinates = self._generate_layout()
        
        # Calculate travel times
        truck_travel_times = self._calculate_truck_travel_times(coordinates)
        robot_travel_times = self._calculate_robot_travel_times(coordinates)
        
        # Generate customer deadlines
        deadlines = self._generate_deadlines(coordinates, truck_travel_times, robot_travel_times)
        
        # Generate customer weights
        weights = self._generate_weights()
        
        # Create customer, drop-off, and depot sets
        C = {f'C{i}' for i in range(self.num_customers)}
        D = {f'D{i}' for i in range(self.num_dropoff)}
        R = {f'R{i}' for i in range(self.num_depots)}
        
        return TBRDInstance(
            C=C,
            D=D,
            R=R,
            deadlines=deadlines,
            weights=weights,
            initial_robots=self.truck_capacity,  # δ = K as specified
            truck_capacity=self.truck_capacity,
            truck_travel_times=truck_travel_times,
            robot_travel_times=robot_travel_times,
            coordinates=coordinates
        )

    def _generate_layout(self) -> Dict[str, Tuple[float, float]]:
        """Generate geographical layout of depots, drop-off points, and customers"""
        coordinates = {}
        
        # Generate equidistant robot depots
        depots_per_side = int(np.ceil(np.sqrt(self.num_depots)))
        spacing = self.w / (depots_per_side + 1)
        
        depot_idx = 0
        for i in range(1, depots_per_side + 1):
            for j in range(1, depots_per_side + 1):
                if depot_idx < self.num_depots:
                    coordinates[f'R{depot_idx}'] = (i * spacing, j * spacing)
                    depot_idx += 1
        
        # Generate random drop-off points
        used_positions = set((x, y) for x, y in coordinates.values())
        for i in range(self.num_dropoff):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'D{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Generate random customer positions
        for i in range(self.num_customers):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'C{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Set initial truck position (γ)
        possible_starts = [k for k in coordinates.keys() if k.startswith(('R', 'D'))]
        gamma = random.choice(possible_starts)
        coordinates['gamma'] = coordinates[gamma]
        
        return coordinates

    def _calculate_euclidean_distance(self, 
                                    p1: Tuple[float, float], 
                                    p2: Tuple[float, float]) -> float:
        """Calculate Euclidean distance between two points"""
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def _calculate_truck_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, str], float]:
        """Calculate truck travel times between all relevant points"""
        travel_times = {}
        locations = [k for k in coordinates.keys() if k != 'gamma_e']
        
        for i in locations:
            for j in locations:
                if i != j:
                    distance = self._calculate_euclidean_distance(
                        coordinates[i], coordinates[j])
                    travel_times[(i, j)] = distance / self.truck_speed
        
        return travel_times

    def _calculate_robot_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, str], float]:
        """Calculate robot travel times from all possible launch points to customers"""
        travel_times = {}
        launch_points = [k for k in coordinates.keys() if k.startswith(('R', 'D', 'gamma'))]
        customers = [k for k in coordinates.keys() if k.startswith('C')]
        
        for i in launch_points:
            for k in customers:
                distance = self._calculate_euclidean_distance(
                    coordinates[i], coordinates[k])
                travel_times[(i, k)] = distance / self.robot_speed
        
        return travel_times

    def _generate_deadlines(self, 
                          coordinates: Dict[str, Tuple[float, float]],
                          truck_times: Dict[Tuple[str, str], float],
                          robot_times: Dict[Tuple[str, str], float]
                          ) -> Dict[str, float]:
        """Generate customer deadlines based on minimum travel times"""
        deadlines = {}
        customers = [k for k in coordinates.keys() if k.startswith('C')]
        
        for k in customers:
            # Calculate minimum travel time to customer k
            min_time = float('inf')
            for j in [p for p in coordinates.keys() if p.startswith(('R', 'D'))]:
                if ('gamma', j) in truck_times and (j, k) in robot_times:
                    total_time = truck_times[('gamma', j)] + robot_times[(j, k)]
                    min_time = min(min_time, total_time)
            
            # Generate random factor and calculate deadline
            r_k = random.uniform(0, 1)
            deadlines[k] = min_time * (self.rho_min + (self.rho_max - self.rho_min) * r_k)
            
        return deadlines

    def _generate_weights(self) -> Dict[str, float]:
        """Generate random customer weights"""
        return {f'C{k}': random.uniform(self.w_min, self.w_max) 
                for k in range(self.num_customers)}

class TBRDHeuristics:
    def __init__(self, instance: TBRDInstance):
        """Initialize heuristic solver with problem instance"""
        self.instance = instance
        self.gamma = 'gamma'
        self.gamma_e = 'gamma_e'
        self._travel_time_cache = {}  # Only cache, no recursive creation
        self._route_cache = {}  # Add route cache
    
    def _get_travel_time(self, from_loc: str, to_loc: str) -> float:
        """Get travel time between locations, handling same-location case"""
        cache_key = (from_loc, to_loc)
        if cache_key not in self._travel_time_cache:
            if from_loc == to_loc:
                self._travel_time_cache[cache_key] = 0.0
            else:
                self._travel_time_cache[cache_key] = self.instance.truck_travel_times.get((from_loc, to_loc), float('inf'))
                if len(self._travel_time_cache) > 1000000:  # Memory management
                    self._travel_time_cache.clear()
        return self._travel_time_cache[cache_key]
    
    def _get_robot_time(self, from_loc: str, to_loc: str) -> float:
        """Get robot travel time between locations, handling same-location case"""
        if from_loc == to_loc:
            return 0.0
        return self.instance.robot_travel_times.get((from_loc, to_loc), float('inf'))
    
    def solve_route(self, route: List[str]) -> Tuple[Optional[Dict[str, str]], float]:
        """
        Given a fixed truck route, find optimal robot launching plan using Hitchcock Transportation Problem
        Returns customer assignments and objective value
        """
        # Add caching at start of method
        route_key = tuple(route)  # Convert list to tuple for hashing
        if hasattr(self, '_route_cache') and route_key in self._route_cache:
            return self._route_cache[route_key]
        
        if not route:
            return None, float('inf')
            
        try:
            # Identify depots and subtours in route
            R_v, S_v = self._identify_route_components(route)
        
            if not any(R_v) and not any(S_v):
                return None, float('inf')
        
            # Create Hitchcock Transportation Problem
            h_model = LpProblem("TBRD_HTP", LpMinimize)
        
            # Create supply nodes
            supply = {}
            # For each depot in route - supply |C| robots
            for i, depot in enumerate(R_v):
                supply[f'r{i}'] = len(self.instance.C)
            
            # For each subtour - supply K robots (or δ for first subtour if γ ∈ R)
            for k, subtour in enumerate(S_v):
                if k == 0 and route[0] in self.instance.R:
                    supply[f's{k}'] = self.instance.initial_robots
                else:
                    supply[f's{k}'] = self.instance.truck_capacity
                
            if not supply:
                return None, float('inf')
            
            # Create demand nodes
            demand = {c: 1 for c in self.instance.C}  # Each customer demands 1 robot
        
            if not demand:  # No customers
                return None, float('inf')
        
            # Add dummy demand node to balance supply/demand
            total_supply = sum(supply.values())
            total_demand = sum(demand.values())
            demand['dummy'] = max(0, total_supply - total_demand)
        
            # Calculate costs before creating variables to reduce problem size
            costs = self._calculate_transportation_costs(route, R_v, S_v)
            if not costs:
                return None, float('inf')
        
            # Only create variables for non-infinite cost assignments
            vars = {}
            for i in supply.keys():
                for j in list(demand.keys()):
                    if j == 'dummy' or costs.get((i,j), float('inf')) < float('inf'):
                        vars[i,j] = LpVariable(f"Route_{i}_{j}", 0, None)
        
            # Add supply constraints (only for created variables)
            for i, amt in supply.items():
                h_model += lpSum(vars[i,j] for j in demand.keys() if (i,j) in vars) == amt
            
            # Add demand constraints (only for created variables)
            for j, amt in demand.items():
                h_model += lpSum(vars[i,j] for i in supply.keys() if (i,j) in vars) == amt
        
            # Set objective (only for created variables)
            h_model += lpSum(vars[i,j] * costs[i,j] 
                            for (i,j) in vars.keys() if (i,j) in costs)
        
            # Solve with tighter tolerance for small problems
            if len(self.instance.C) <= 10:
                status = h_model.solve(PULP_CBC_CMD(msg=0, gapRel=0.001))
            else:
                status = h_model.solve(PULP_CBC_CMD(msg=0))
        
            if status != LpStatusOptimal:
                return None, float('inf')
        
            # Extract solution
            assignments = {}
            for i in supply.keys():
                for j in list(demand.keys()):
                    if (i,j) in vars and j != 'dummy' and vars[i,j].value() is not None and vars[i,j].value() > 0.5:
                        if i.startswith('r'):  # From depot
                            depot_idx = int(i[1:])
                            if depot_idx < len(R_v):
                                assignments[j] = R_v[depot_idx]
                        else:  # From subtour
                            subtour_idx = int(i[1:])
                            if subtour_idx < len(S_v):
                                best_drop = self._get_best_dropoff(j, S_v[subtour_idx])
                                if best_drop:
                                    assignments[j] = best_drop
        
            if not assignments:
                return None, float('inf')
            
            # Store in cache before returning
            if not hasattr(self, '_route_cache'):
                self._route_cache = {}
            if len(self._route_cache) > 1000:  # Memory management
                self._route_cache.clear()
            result = (assignments, value(h_model.objective))
            self._route_cache[route_key] = result
            return result
        
        except Exception as e:
            print(f"\nError in solve_route: {str(e)}")
            return None, float('inf')
    
    def _identify_route_components(self, route: List[str]) -> Tuple[List[str], List[List[str]]]:
        """Identify depots R(v) and subtours S(v) in given route"""
        R_v = []  # Depots in route
        S_v = []  # Subtours (sequences of drop-off points)
        current_subtour = []
        
        for i, loc in enumerate(route):
            if loc in self.instance.R:
                R_v.append(loc)
                if current_subtour:
                    S_v.append(current_subtour)
                    current_subtour = []
            elif loc in self.instance.D:
                current_subtour.append(loc)
                
        if current_subtour:
            S_v.append(current_subtour)
            
        return R_v, S_v
    
    def _calculate_transportation_costs(self, route: List[str], 
                                     R_v: List[str],
                                     S_v: List[List[str]]) -> Dict[Tuple[str,str], float]:
        """Calculate costs for Hitchcock Transportation Problem"""
        try:
            costs = {}
            current_time = 0
            
            # Calculate arrival times at each location
            arrival_times = {route[0]: 0}
            for i in range(1, len(route)):
                current_time += self._get_travel_time(route[i-1], route[i])
                arrival_times[route[i]] = current_time
                
            # Calculate costs from depots
            for i, depot in enumerate(R_v):
                depot_time = arrival_times.get(depot, 0)
                for cust in self.instance.C:
                    total_time = depot_time + self._get_robot_time(depot, cust)
                    costs[f'r{i}', cust] = (self.instance.weights[cust] 
                                          if total_time > self.instance.deadlines[cust] 
                                          else 0)
                                          
            # Calculate costs from subtours
            for k, subtour in enumerate(S_v):
                for cust in self.instance.C:
                    # Find best (earliest) drop-off point in subtour
                    min_time = float('inf')
                    for drop in subtour:
                        drop_time = arrival_times.get(drop, 0)
                        total_time = drop_time + self._get_robot_time(drop, cust)
                        min_time = min(min_time, total_time)
                        
                    costs[f's{k}', cust] = (self.instance.weights[cust]
                                         if min_time > self.instance.deadlines[cust]
                                         else 0)
                    
            # Set costs to dummy node as 0
            for i in [f'r{i}' for i in range(len(R_v))] + [f's{i}' for i in range(len(S_v))]:
                costs[i, 'dummy'] = 0
                
            return costs
        except Exception as e:
            print(f"\nError in calculate_transportation_costs: {str(e)}")
            return {}
    
    def _get_best_dropoff(self, customer: str, subtour: List[str]) -> Optional[str]:
        """Find best drop-off point in subtour for given customer"""
        if not subtour:
            return None
            
        try:
            best_point = min(subtour,
                           key=lambda drop: self._get_robot_time(drop, customer))
            return best_point
        except Exception as e:
            print(f"\nError in get_best_dropoff: {str(e)}")
            return None

class TBRDModel:
    def __init__(self, instance: TBRDInstance):
        """Initialize TBRD model with problem instance"""
        self.instance = instance
        self.gamma = 'gamma'
        self.gamma_e = 'gamma_e'
        self.heuristic_solver = TBRDHeuristics(instance)
        
    def _get_travel_time(self, from_loc: str, to_loc: str) -> float:
        """Get travel time between locations, handling same-location case"""
        if from_loc == to_loc:
            return 0.0
        return self.instance.truck_travel_times.get((from_loc, to_loc), float('inf'))

    def solve(self, time_limit: Optional[int] = None, gap: Optional[float] = None) -> int:
        """Multi-start local search procedure from section 4.3"""
        try:
            self._start_time = time.time()
        
            # Step 1: Generate initial solutions using both priority rules
            route_PR1 = self._generate_route_PR1()
            if route_PR1:
                solution_PR1, obj_PR1 = self.heuristic_solver.solve_route(route_PR1)
                # Add early termination
                if obj_PR1 == 0:  
                    self.final_route = route_PR1
                    self.assignments = solution_PR1
                    self.objective_value = obj_PR1
                    self._solve_time = time.time() - self._start_time
                    return LpStatusOptimal
            else:
                solution_PR1, obj_PR1 = None, float('inf')
        
            route_PR2 = self._generate_route_PR2()
            if route_PR2:
                solution_PR2, obj_PR2 = self.heuristic_solver.solve_route(route_PR2)
                # Add early termination
                if obj_PR2 == 0:
                    self.final_route = route_PR2
                    self.assignments = solution_PR2
                    self.objective_value = obj_PR2
                    self._solve_time = time.time() - self._start_time
                    return LpStatusOptimal
            else:
                solution_PR2, obj_PR2 = None, float('inf')
            
            # Step 2: Generate solution pools using depot insertions
            pool_PR1 = self._generate_solution_pool(route_PR1, obj_PR1) if route_PR1 else []
            pool_PR2 = self._generate_solution_pool(route_PR2, obj_PR2) if route_PR2 else []
            
            # Step 3: Local search on both pools
            best_solution = None
            best_objective = float('inf')
            
            # Use both pools
            for pool_idx, pool in enumerate([pool_PR1, pool_PR2]):
                if pool:  # Only process non-empty pools
                    for _ in range(2):  # κrs/2 times per pool as per paper
                        local_solution = self._local_search(pool.copy())
                        if local_solution:
                            route, assignments, obj = local_solution
                            if obj < best_objective:
                                best_objective = obj
                                best_solution = (route, assignments, obj)
            
            if best_solution:
                self.final_route, self.assignments, self.objective_value = best_solution
                self._solve_time = time.time() - self._start_time
                return LpStatusOptimal
            
            return LpStatusNotSolved
            
        except Exception as e:
            print(f"\nERROR: Solver failed with error: {str(e)}")
            return LpStatusNotSolved
    
    def _generate_route_PR1(self) -> Optional[List[str]]:
        """PR1: Move to position with most satisfiable customers"""
        try:
            route = [self.gamma]
            processed_customers = set()
            current_pos = self.gamma
            current_time = 0
            
            iteration = 0
            while len(processed_customers) < len(self.instance.C) and iteration < 100:  # Added iteration limit
                iteration += 1
                
                # Get customers satisfiable from current position
                satisfiable = {
                    k for k in (self.instance.C - processed_customers)
                    if current_time + self.instance.robot_travel_times.get((current_pos, k), float('inf')) <= self.instance.deadlines[k]
                }
                processed_customers.update(satisfiable)
                
                # Find location with most satisfiable unprocessed customers
                best_location = None
                max_satisfiable = -1
                
                remaining_locations = (self.instance.D.union(self.instance.R)) - set(route)
                
                if not remaining_locations:
                    if processed_customers == self.instance.C:
                        return route
                    else:
                        return None
                
                for loc in remaining_locations:
                    next_time = current_time + self._get_travel_time(current_pos, loc)
                    count = sum(1 for k in (self.instance.C - processed_customers)
                              if next_time + self.instance.robot_travel_times.get((loc, k), float('inf')) <= self.instance.deadlines[k])
                    if count > max_satisfiable:
                        max_satisfiable = count
                        best_location = loc
                
                if best_location is None:
                    unvisited_depots = self.instance.R - set(route)
                    if unvisited_depots:
                        best_location = min(unvisited_depots,
                                          key=lambda r: self._get_travel_time(current_pos, r))
                    else:
                        return None
                
                route.append(best_location)
                current_pos = best_location
                current_time += self._get_travel_time(route[-2], route[-1])
                
                if len(route) > len(self.instance.C) + len(self.instance.D) + len(self.instance.R):
                    return None
            
            return route
            
        except Exception as e:
            print(f"\nError in generate_route_PR1: {str(e)}")
            return None

    def _generate_route_PR2(self) -> Optional[List[str]]:
        """PR2: Move to position with highest urgency"""
        try:
            route = [self.gamma]
            processed_customers = set()
            current_pos = self.gamma
            current_time = 0
            
            # Assign each customer to nearest depot or drop-off point
            nearest_point = {}
            for k in self.instance.C:
                options = self.instance.D.union(self.instance.R)
                if options:
                    nearest = min(options,
                                key=lambda p: self.instance.robot_travel_times.get((p, k), float('inf')))
                    nearest_point[k] = nearest
                else:
                    return None
            
            iteration = 0
            while len(processed_customers) < len(self.instance.C) and iteration < 100:  # Added iteration limit
                iteration += 1
                
                best_location = None
                min_urgency = float('inf')
                
                remaining_locations = (self.instance.D.union(self.instance.R)) - set(route)
                
                if not remaining_locations:
                    if processed_customers == self.instance.C:
                        return route
                    else:
                        return None
                
                for loc in remaining_locations:
                    next_time = current_time + self._get_travel_time(current_pos, loc)
                    assigned_customers = {k for k, p in nearest_point.items() 
                                       if p == loc and k not in processed_customers}
                    
                    if assigned_customers:
                        min_budget = min(self.instance.deadlines[k] - 
                                       (next_time + self.instance.robot_travel_times.get((loc, k), float('inf')))
                                       for k in assigned_customers)
                        if min_budget < min_urgency:
                            min_urgency = min_budget
                            best_location = loc
                
                if best_location is None:
                    unvisited_depots = self.instance.R - set(route)
                    if unvisited_depots:
                        best_location = min(unvisited_depots,
                                          key=lambda r: self._get_travel_time(current_pos, r))
                    else:
                        return None
                
                route.append(best_location)
                current_pos = best_location
                current_time += self._get_travel_time(route[-2], route[-1])
                
                # Update processed customers
                for k in set(self.instance.C) - processed_customers:
                    if nearest_point.get(k) == best_location:
                        if current_time + self.instance.robot_travel_times.get((best_location, k), float('inf')) <= self.instance.deadlines[k]:
                            processed_customers.add(k)
                
                if len(route) > len(self.instance.C) + len(self.instance.D) + len(self.instance.R):
                    return None
            
            return route
            
        except Exception as e:
            print(f"\nError in generate_route_PR2: {str(e)}")
            return None
    
    def _generate_solution_pool(self, initial_route: Optional[List[str]], initial_obj: float) -> List[Tuple[List[str], float]]:
        """Generate pool of solutions using depot insertions"""
        if initial_route is None or initial_obj == float('inf'):
            return []
            
        pool = [(initial_route, initial_obj)]
        
        try:
            # Find consecutive drop-off points in route
            for i in range(len(initial_route) - 1):
                if (initial_route[i] in self.instance.D and 
                    initial_route[i+1] in self.instance.D):
                    
                    # Try inserting each depot between them
                    for depot in self.instance.R:
                        if depot not in initial_route[i:i+2]:
                            new_route = initial_route[:i+1] + [depot] + initial_route[i+1:]
                            assignments, obj = self.heuristic_solver.solve_route(new_route)
                            
                            if assignments is not None and obj != float('inf') and obj < initial_obj:
                                pool.append((new_route, obj))
            
            return pool
        except Exception as e:
            print(f"\nError in generate_solution_pool: {str(e)}")
            return [(initial_route, initial_obj)]

    def _local_search(self, pool: List[Tuple[List[str], float]], 
                     iterations_ls1: int = 100,
                     iterations_ls2: int = 200) -> Optional[Tuple[List[str], Dict[str, str], float]]:
        """Local search procedure"""
        if not pool:
            return None
    
        try:
            # Filter out any invalid solutions from pool
            valid_pool = [(route, obj) for route, obj in pool if route and obj != float('inf')]
            if not valid_pool:
                return None
                
            current_solution = min(valid_pool, key=lambda x: x[1])  # Get solution with lowest objective
            current_route, current_obj = current_solution
    
            if current_route is None or current_obj == float('inf'):
                return None

            # Early termination if optimal
            if current_obj == 0:
                current_assignments, _ = self.heuristic_solver.solve_route(current_route)
                return current_route, current_assignments, current_obj
                
            current_assignments, _ = self.heuristic_solver.solve_route(current_route)
            if current_assignments is None:
                return None
            
            # First phase with adaptive iterations
            no_improvement_count = 0
            for iter in range(iterations_ls1):
                if no_improvement_count >= 20:  # Stop if no improvement for 20 iterations
                    break
                
                neighborhood = random.choice(['N1', 'N2', 'N3', 'N4'])
                new_route = self._apply_neighborhood(current_route, neighborhood)
            
                if new_route:
                    assignments, obj = self.heuristic_solver.solve_route(new_route)
                    if assignments is not None and obj != float('inf') and obj < current_obj:
                        current_route = new_route
                        current_obj = obj
                        current_assignments = assignments
                        valid_pool = [(r, o) for r, o in valid_pool if o < current_obj]
                        no_improvement_count = 0
                    else:
                        no_improvement_count += 1
        
            # Second phase with adaptive iterations
            no_improvement_count = 0
            for iter in range(iterations_ls2):
                if no_improvement_count >= 20:
                    break
                
                neighborhood = random.choice(['N1', 'N2', 'N3', 'N4'])
                new_route = self._apply_neighborhood(current_route, neighborhood)
            
                if new_route:
                    assignments, obj = self.heuristic_solver.solve_route(new_route)
                    if assignments is not None and obj != float('inf') and obj < current_obj:
                        current_route = new_route
                        current_obj = obj
                        current_assignments = assignments
                        no_improvement_count = 0
                    else:
                        no_improvement_count += 1
        
            return current_route, current_assignments, current_obj
            
        except Exception as e:
            print(f"\nError in local_search: {str(e)}")
            return None
    
    def _apply_neighborhood(self, route: List[str], neighborhood: str) -> Optional[List[str]]:
        """Apply neighborhood operation to route"""
        # Early pruning checks
        if not route or len(route) < 2:
            return None
        
        if len(route) > len(self.instance.C) + len(self.instance.D) + len(self.instance.R):
            return None
        
        # Check if route contains required minimum elements
        if not any(loc in self.instance.R for loc in route):  # Must have at least one depot
            return None
        
        if not any(loc in self.instance.D for loc in route):  # Must have at least one drop-off
            return None

        try:
            if neighborhood == 'N1':
                # Remove random position (not single depot)
                depot_counts = sum(1 for loc in route if loc in self.instance.R)
                if depot_counts > 1:
                    # Only try to remove non-critical positions
                    removable_indices = [i for i in range(1, len(route)) 
                                       if not (route[i] in self.instance.R and depot_counts <= 1)]
                    if removable_indices:
                        idx = random.choice(removable_indices)
                        new_route = route[:idx] + route[idx+1:]
                        if len(new_route) >= 2:  # Ensure route still valid after removal
                            return new_route
                    
            elif neighborhood == 'N2':
                # Insert random depot/drop-off, but be smarter about insertion
                remaining_locations = self.instance.D.union(self.instance.R) - set(route)
                if remaining_locations:  # Only insert if we have unused locations
                    location = random.choice(list(remaining_locations))
                    # Don't insert between consecutive drop-offs if it's a depot
                    valid_positions = [i for i in range(1, len(route)) 
                                     if not (location in self.instance.R and 
                                           i > 0 and i < len(route) and
                                           route[i-1] in self.instance.D and 
                                           route[i] in self.instance.D)]
                    if valid_positions:
                        idx = random.choice(valid_positions)
                        return route[:idx] + [location] + route[idx:]
            
            elif neighborhood == 'N3':
                # Swap two positions, but be smarter about which positions to swap
                valid_indices = range(1, len(route))
                if len(valid_indices) >= 2:
                    idx1, idx2 = random.sample(list(valid_indices), 2)
                    # Don't swap if it would create invalid depot sequence
                    if not (route[idx1] in self.instance.R and route[idx2] in self.instance.R):
                        new_route = route.copy()
                        new_route[idx1], new_route[idx2] = new_route[idx2], new_route[idx1]
                        return new_route
                
            elif neighborhood == 'N4':
                # Insert closest depot before drop-off point
                drop_positions = [i for i, loc in enumerate(route) if loc in self.instance.D]
                unvisited_depots = self.instance.R - set(route)
                if drop_positions and unvisited_depots:
                    pos = random.choice(drop_positions)
                    # Find depot that minimizes total travel time
                    nearest_depot = min(unvisited_depots,
                                     key=lambda r: (self._get_travel_time(route[pos-1], r) +
                                                  self._get_travel_time(r, route[pos])))
                    return route[:pos] + [nearest_depot] + route[pos:]
        
            return None
        
        except Exception as e:
            print(f"\nError in apply_neighborhood: {str(e)}")
            return None
    
    def get_solution(self) -> Optional[Dict]:
        """Return the solution details"""
        if not hasattr(self, 'objective_value') or self.objective_value is None:
            return None
            
        try:
            solution = {
                'objective_value': float(self.objective_value),
                'route': self.final_route,
                'customer_assignments': self.assignments,
                'coordinates': self.instance.coordinates,
                'completion_time': getattr(self, '_solve_time', None),
                'is_optimal': False  # Since we're using heuristic
            }
            return solution
            
        except Exception as e:
            print(f"\nError extracting solution: {str(e)}")
            return None
        
def run_experiment(instance_params: Dict, 
                  num_instances: int = 1,
                  time_limit: Optional[int] = None,
                  gap: Optional[float] = None) -> List[Dict]:
    """
    Run experiments with specified parameters
    """
    results = []
    best_objective = float('inf')
    best_solutions_count = 0    # Count of solutions matching the best objective
    optima_found = 0           # Count of optimal solutions (obj = 0)
    best_solutions_total_time = 0
    
    generator = TBRDInstanceGenerator(**instance_params)
    
    for i in range(num_instances):
        instance = generator.generate_instance(seed=i)
        model = TBRDModel(instance)
        status = model.solve(time_limit=time_limit, gap=gap)
        
        solution = model.get_solution()
        if solution:
            solution['instance_id'] = i
            solution['status'] = status
            results.append(solution)
            
            obj_val = float(solution['objective_value'])
            
            # Check for optimal solution (objective = 0)
            if obj_val == 0:
                optima_found += 1
            
            # Track best solutions
            if obj_val < best_objective:
                best_objective = obj_val
                best_solutions_count = 1
                best_solutions_total_time = solution['completion_time']
            elif obj_val == best_objective:
                best_solutions_count += 1
                best_solutions_total_time += solution['completion_time']
    
    # Print only summary statistics
    if results:
        print("\n" + "="*50)
        print("FINAL SUMMARY STATISTICS")
        print("="*50)
        print(f"Total instances solved: {len(results)}")
        print(f"\nBest Solution Statistics:")
        print(f"Best objective value found: {best_objective}")
        print(f"Number of times best solution was found: {best_solutions_count}")
        print(f"Number of optima found (obj = 0): {optima_found}")
        if best_solutions_count > 0:
            print(f"Average CPU time for best solutions: {best_solutions_total_time/best_solutions_count:.2f} seconds")
        
        print(f"\nOverall Statistics:")
        print(f"Average objective value: {sum(float(r['objective_value']) for r in results) / len(results):.2f}")
        print(f"Average solution time: {sum(r['completion_time'] for r in results) / len(results):.2f} seconds")
    else:
        print("\nNo solutions found.")
    
    return results        
        
if __name__ == "__main__":
    # Medium-sized instance with more relaxed parameters
    instance_params = {
        'w': 2,                # 5 km square area (increased from 2)
        'num_customers': 6,   # 10 customers
        'num_dropoff': 6,      # 5 drop-off points
        'num_depots': 4,       # 3 robot depots
        'truck_speed': 30,     # 40 km/h (increased from 30)
        'robot_speed': 5,     # 20 km/h (increased from 15)
        'truck_capacity': 2,   # Maximum 5 robots (increased from 3)
        'rho_min': 3.0,        # Minimum deadline factor (increased from 1.2)
        'rho_max': 5.0,        # Maximum deadline factor (increased from 2.0)
        'w_min': 1,           # Minimum customer weight
        'w_max': 1            # Maximum customer weight
    }
    
    # Run experiment with two instances
    results = run_experiment(
        instance_params,
        num_instances=25,      # Two instances
        time_limit=200,       # 5 minutes per instance
        gap=0.1              # 10% gap tolerance
    )


Error in generate_solution_pool: '<' not supported between instances of 'NoneType' and 'NoneType'

ERROR: Solver failed with error: '<' not supported between instances of 'NoneType' and 'float'

FINAL SUMMARY STATISTICS
Total instances solved: 24

Best Solution Statistics:
Best objective value found: 0.0
Number of times best solution was found: 24
Number of optima found (obj = 0): 24
Average CPU time for best solutions: 0.13 seconds

Overall Statistics:
Average objective value: 0.00
Average solution time: 0.13 seconds


In [ ]:
import numpy as np
import random
from dataclasses import dataclass
import multiprocessing
import psutil
import time
from typing import List, Dict, Tuple, Set, Optional, Union
from pulp import PULP_CBC_CMD
import signal
from multiprocessing import Process
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FutureTimeoutError

@dataclass
class TBRDInstance:
    """Class to hold a TBRD problem instance"""
    C: Set[str]  # Customers
    D: Set[str]  # Drop-off points
    R: Set[str]  # Robot depots
    deadlines: Dict[str, float]  # Customer deadlines
    weights: Dict[str, float]    # Customer weights
    initial_robots: int          # Number of robots initially loaded (δ)
    truck_capacity: int          # Maximum loading capacity for robots (K)
    truck_travel_times: Dict[Tuple[str, str], float]  # Travel times for truck
    robot_travel_times: Dict[Tuple[str, str], float]  # Travel times for robots
    coordinates: Dict[str, Tuple[float, float]]       # Coordinates of all points

class TBRDInstanceGenerator:
    def __init__(self, 
                 w: float,                # Square side length (km)
                 num_customers: int,      # Number of customers
                 num_dropoff: int,        # Number of drop-off points
                 num_depots: int,         # Number of robot depots
                 truck_speed: float,      # Truck speed (km/h)
                 robot_speed: float,      # Robot speed (km/h)
                 truck_capacity: int,     # Truck capacity (K)
                 rho_min: float,          # Minimum deadline factor
                 rho_max: float,          # Maximum deadline factor
                 w_min: float,            # Minimum customer weight
                 w_max: float):           # Maximum customer weight
        self.w = w
        self.num_customers = num_customers
        self.num_dropoff = num_dropoff
        self.num_depots = num_depots
        self.truck_speed = truck_speed
        self.robot_speed = robot_speed
        self.truck_capacity = truck_capacity
        self.rho_min = rho_min
        self.rho_max = rho_max
        self.w_min = w_min
        self.w_max = w_max
        self.grid_size = 1/6  # As specified in the paper

    def generate_instance(self, seed: Optional[int] = None) -> TBRDInstance:
        """Generate a single TBRD instance"""
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)
            
        # Generate coordinates for all points
        coordinates = self._generate_layout()
        
        # Calculate travel times
        truck_travel_times = self._calculate_truck_travel_times(coordinates)
        robot_travel_times = self._calculate_robot_travel_times(coordinates)
        
        # Generate customer deadlines
        deadlines = self._generate_deadlines(coordinates, truck_travel_times, robot_travel_times)
        
        # Generate customer weights
        weights = self._generate_weights()
        
        # Create customer, drop-off, and depot sets
        C = {f'C{i}' for i in range(self.num_customers)}
        D = {f'D{i}' for i in range(self.num_dropoff)}
        R = {f'R{i}' for i in range(self.num_depots)}
        
        return TBRDInstance(
            C=C,
            D=D,
            R=R,
            deadlines=deadlines,
            weights=weights,
            initial_robots=self.truck_capacity,  # δ = K as specified
            truck_capacity=self.truck_capacity,
            truck_travel_times=truck_travel_times,
            robot_travel_times=robot_travel_times,
            coordinates=coordinates
        )

    def _generate_layout(self) -> Dict[str, Tuple[float, float]]:
        """Generate geographical layout of depots, drop-off points, and customers"""
        coordinates = {}
        
        # Generate equidistant robot depots
        depots_per_side = int(np.ceil(np.sqrt(self.num_depots)))
        spacing = self.w / (depots_per_side + 1)
        
        depot_idx = 0
        for i in range(1, depots_per_side + 1):
            for j in range(1, depots_per_side + 1):
                if depot_idx < self.num_depots:
                    coordinates[f'R{depot_idx}'] = (i * spacing, j * spacing)
                    depot_idx += 1
        
        # Generate random drop-off points
        used_positions = set((x, y) for x, y in coordinates.values())
        for i in range(self.num_dropoff):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'D{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Generate random customer positions
        for i in range(self.num_customers):
            while True:
                x = random.uniform(0, self.w)
                y = random.uniform(0, self.w)
                pos = (round(x/self.grid_size)*self.grid_size, 
                      round(y/self.grid_size)*self.grid_size)
                if pos not in used_positions:
                    coordinates[f'C{i}'] = pos
                    used_positions.add(pos)
                    break
        
        # Set initial truck position (γ)
        possible_starts = [k for k in coordinates.keys() if k.startswith(('R', 'D'))]
        gamma = random.choice(possible_starts)
        coordinates['gamma'] = coordinates[gamma]
        
        return coordinates

    def _calculate_euclidean_distance(self, 
                                    p1: Tuple[float, float], 
                                    p2: Tuple[float, float]) -> float:
        """Calculate Euclidean distance between two points"""
        return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    def _calculate_truck_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, str], float]:
        """Calculate truck travel times between all relevant points"""
        travel_times = {}
        locations = [k for k in coordinates.keys() if k != 'gamma_e']
        
        for i in locations:
            for j in locations:
                if i != j:
                    distance = self._calculate_euclidean_distance(
                        coordinates[i], coordinates[j])
                    travel_times[(i, j)] = distance / self.truck_speed
        
        return travel_times

    def _calculate_robot_travel_times(self, 
                                    coordinates: Dict[str, Tuple[float, float]]
                                    ) -> Dict[Tuple[str, str], float]:
        """Calculate robot travel times from all possible launch points to customers"""
        travel_times = {}
        launch_points = [k for k in coordinates.keys() if k.startswith(('R', 'D', 'gamma'))]
        customers = [k for k in coordinates.keys() if k.startswith('C')]
        
        for i in launch_points:
            for k in customers:
                distance = self._calculate_euclidean_distance(
                    coordinates[i], coordinates[k])
                travel_times[(i, k)] = distance / self.robot_speed
        
        return travel_times

    def _generate_deadlines(self, 
                          coordinates: Dict[str, Tuple[float, float]],
                          truck_times: Dict[Tuple[str, str], float],
                          robot_times: Dict[Tuple[str, str], float]
                          ) -> Dict[str, float]:
        """Generate customer deadlines based on minimum travel times"""
        deadlines = {}
        customers = [k for k in coordinates.keys() if k.startswith('C')]
        
        for k in customers:
            # Calculate minimum travel time to customer k
            min_time = float('inf')
            for j in [p for p in coordinates.keys() if p.startswith(('R', 'D'))]:
                if ('gamma', j) in truck_times and (j, k) in robot_times:
                    total_time = truck_times[('gamma', j)] + robot_times[(j, k)]
                    min_time = min(min_time, total_time)
            
            # Generate random factor and calculate deadline
            r_k = random.uniform(0, 1)
            deadlines[k] = min_time * (self.rho_min + (self.rho_max - self.rho_min) * r_k)
            
        return deadlines

    def _generate_weights(self) -> Dict[str, float]:
        """Generate random customer weights"""
        return {f'C{k}': random.uniform(self.w_min, self.w_max) 
                for k in range(self.num_customers)}

class TBRDHeuristics:
    def __init__(self, instance: TBRDInstance):
        """Initialize heuristic solver with problem instance"""
        self.instance = instance
        self.gamma = 'gamma'
        self.gamma_e = 'gamma_e'
        self._travel_time_cache = {}  # Only cache, no recursive creation
        self._route_cache = {}  # Add route cache
    
    def _get_travel_time(self, from_loc: str, to_loc: str) -> float:
        """Get travel time between locations, handling same-location case"""
        cache_key = (from_loc, to_loc)
        if cache_key not in self._travel_time_cache:
            if from_loc == to_loc:
                self._travel_time_cache[cache_key] = 0.0
            else:
                self._travel_time_cache[cache_key] = self.instance.truck_travel_times.get((from_loc, to_loc), float('inf'))
                if len(self._travel_time_cache) > 1000000:  # Memory management
                    self._travel_time_cache.clear()
        return self._travel_time_cache[cache_key]
    
    def _get_robot_time(self, from_loc: str, to_loc: str) -> float:
        """Get robot travel time between locations, handling same-location case"""
        if from_loc == to_loc:
            return 0.0
        return self.instance.robot_travel_times.get((from_loc, to_loc), float('inf'))
    
    def solve_route(self, route: List[str]) -> Tuple[Optional[Dict[str, str]], float]:
        """
        Given a fixed truck route, find optimal robot launching plan using Hitchcock Transportation Problem
        Returns customer assignments and objective value
        """
        # Add caching at start of method
        route_key = tuple(route)  # Convert list to tuple for hashing
        if route_key in self._route_cache:
            return self._route_cache[route_key]
        
        if not route:
            return None, float('inf')
            
        try:
            # Identify depots and subtours in route
            R_v, S_v = self._identify_route_components(route)
        
            if not any(R_v) and not any(S_v):
                return None, float('inf')
        
            # Create Hitchcock Transportation Problem
            h_model = LpProblem("TBRD_HTP", LpMinimize)
        
            # Create supply nodes
            supply = {}
            # For each depot in route - supply |C| robots
            for i, depot in enumerate(R_v):
                supply[f'r{i}'] = len(self.instance.C)
            
            # For each subtour - supply K robots (or δ for first subtour if γ ∈ R)
            for k, subtour in enumerate(S_v):
                if k == 0 and route[0] in self.instance.R:
                    supply[f's{k}'] = self.instance.initial_robots
                else:
                    supply[f's{k}'] = self.instance.truck_capacity
                
            if not supply:
                return None, float('inf')
            
            # Create demand nodes
            demand = {c: 1 for c in self.instance.C}  # Each customer demands 1 robot
        
            if not demand:  # No customers
                return None, float('inf')
        
            # Add dummy demand node to balance supply/demand
            total_supply = sum(supply.values())
            total_demand = sum(demand.values())
            demand['dummy'] = max(0, total_supply - total_demand)
        
            # Calculate costs before creating variables to reduce problem size
            costs = self._calculate_transportation_costs(route, R_v, S_v)
            if not costs:
                return None, float('inf')
            
            # Before creating variables, pre-check which assignments are feasible
            feasible_assignments = []
            for i in supply.keys():
                for j in list(demand.keys()):
                    if j == 'dummy' or costs.get((i,j), float('inf')) == 0:  # Only create variables for zero-cost assignments
                        feasible_assignments.append((i,j))
                    
            # Only create variables for feasible assignments
            vars = {(i,j): LpVariable(f"Route_{i}_{j}", 0, None) 
                    for (i,j) in feasible_assignments}
        
            # Add supply constraints (only for created variables)
            for i, amt in supply.items():
                h_model += lpSum(vars[i,j] for j in demand.keys() if (i,j) in vars) == amt
            
            # Add demand constraints (only for created variables)
            for j, amt in demand.items():
                h_model += lpSum(vars[i,j] for i in supply.keys() if (i,j) in vars) == amt
        
            # Set objective (only for created variables)
            h_model += lpSum(vars[i,j] * costs[i,j] 
                            for (i,j) in vars.keys() if (i,j) in costs)
        
            # Solve with tighter tolerance for small problems
            if len(self.instance.C) <= 10:
                status = h_model.solve(PULP_CBC_CMD(msg=0, gapRel=0.001))
            else:
                status = h_model.solve(PULP_CBC_CMD(msg=0))
        
            if status != LpStatusOptimal:
                return None, float('inf')
        
            # Extract solution
            assignments = {}
            for i in supply.keys():
                for j in list(demand.keys()):
                    if (i,j) in vars and j != 'dummy' and vars[i,j].value() is not None and vars[i,j].value() > 0.5:
                        if i.startswith('r'):  # From depot
                            depot_idx = int(i[1:])
                            if depot_idx < len(R_v):
                                assignments[j] = R_v[depot_idx]
                        else:  # From subtour
                            subtour_idx = int(i[1:])
                            if subtour_idx < len(S_v):
                                best_drop = self._get_best_dropoff(j, S_v[subtour_idx])
                                if best_drop:
                                    assignments[j] = best_drop
        
            if not assignments:
                return None, float('inf')
            
            # Store in cache before returning
            if len(self._route_cache) > 1000:  # Memory management
                self._route_cache.clear()
            result = (assignments, value(h_model.objective))
            self._route_cache[route_key] = result
            return result
        
        except Exception as e:
            print(f"\nError in solve_route: {str(e)}")
            return None, float('inf')
    
    def _identify_route_components(self, route: List[str]) -> Tuple[List[str], List[List[str]]]:
        """Identify depots R(v) and subtours S(v) in given route"""
        R_v = []  # Depots in route
        S_v = []  # Subtours (sequences of drop-off points)
        current_subtour = []
        
        for i, loc in enumerate(route):
            if loc in self.instance.R:
                R_v.append(loc)
                if current_subtour:
                    S_v.append(current_subtour)
                    current_subtour = []
            elif loc in self.instance.D:
                current_subtour.append(loc)
                
        if current_subtour:
            S_v.append(current_subtour)
            
        return R_v, S_v
    
    def _calculate_transportation_costs(self, route: List[str], 
                                     R_v: List[str],
                                     S_v: List[List[str]]) -> Dict[Tuple[str,str], float]:
        """Calculate costs for Hitchcock Transportation Problem"""
        try:
            costs = {}
        
            # Add arrival times caching
            if not hasattr(self, '_arrival_times_cache'):
                self._arrival_times_cache = {}
            route_key = tuple(route)
        
            if route_key in self._arrival_times_cache:
                arrival_times = self._arrival_times_cache[route_key]
            else:
                arrival_times = {route[0]: 0}
                current_time = 0
                for i in range(1, len(route)):
                    current_time += self._get_travel_time(route[i-1], route[i])
                    arrival_times[route[i]] = current_time
                if len(self._arrival_times_cache) > 1000:
                    self._arrival_times_cache.clear()
                self._arrival_times_cache[route_key] = arrival_times
                
            # Calculate costs from depots
            for i, depot in enumerate(R_v):
                depot_time = arrival_times.get(depot, 0)
                for cust in self.instance.C:
                    total_time = depot_time + self._get_robot_time(depot, cust)
                    costs[f'r{i}', cust] = (self.instance.weights[cust] 
                                          if total_time > self.instance.deadlines[cust] 
                                          else 0)
                                      
            # Calculate costs from subtours
            for k, subtour in enumerate(S_v):
                for cust in self.instance.C:
                    # Find best (earliest) drop-off point in subtour
                    min_time = float('inf')
                    for drop in subtour:
                        drop_time = arrival_times.get(drop, 0)
                        total_time = drop_time + self._get_robot_time(drop, cust)
                        min_time = min(min_time, total_time)
                    
                    costs[f's{k}', cust] = (self.instance.weights[cust]
                                         if min_time > self.instance.deadlines[cust]
                                         else 0)
                
            # Set costs to dummy node as 0
            for i in [f'r{i}' for i in range(len(R_v))] + [f's{i}' for i in range(len(S_v))]:
                costs[i, 'dummy'] = 0
            
            return costs
        except Exception as e:
            print(f"\nError in calculate_transportation_costs: {str(e)}")
            return {}
    
    def _get_best_dropoff(self, customer: str, subtour: List[str]) -> Optional[str]:
        """Find best drop-off point in subtour for given customer"""
        if not subtour:
            return None
            
        try:
            best_point = min(subtour,
                           key=lambda drop: self._get_robot_time(drop, customer))
            return best_point
        except Exception as e:
            print(f"\nError in get_best_dropoff: {str(e)}")
            return None

class TBRDModel:
    def __init__(self, instance: TBRDInstance):
        """Initialize TBRD model with problem instance"""
        self.instance = instance
        self.gamma = 'gamma'
        self.gamma_e = 'gamma_e'
        self.heuristic_solver = TBRDHeuristics(instance)
        
    def _get_travel_time(self, from_loc: str, to_loc: str) -> float:
        """Get travel time between locations, handling same-location case"""
        if from_loc == to_loc:
            return 0.0
        return self.instance.truck_travel_times.get((from_loc, to_loc), float('inf'))

    def solve(self, time_limit: Optional[int] = None, gap: Optional[float] = None) -> int:
        """Multi-start local search procedure from section 4.3"""
        try:
            self._start_time = time.time()
        
            # Step 1: Generate initial solutions using both priority rules
            route_PR1 = self._generate_route_PR1()
            if route_PR1:
                solution_PR1, obj_PR1 = self.heuristic_solver.solve_route(route_PR1)
                # Add early termination
                if obj_PR1 == 0:  
                    self.final_route = route_PR1
                    self.assignments = solution_PR1
                    self.objective_value = obj_PR1
                    self._solve_time = time.time() - self._start_time
                    return LpStatusOptimal
            else:
                solution_PR1, obj_PR1 = None, float('inf')
        
            route_PR2 = self._generate_route_PR2()
            if route_PR2:
                solution_PR2, obj_PR2 = self.heuristic_solver.solve_route(route_PR2)
                # Add early termination
                if obj_PR2 == 0:
                    self.final_route = route_PR2
                    self.assignments = solution_PR2
                    self.objective_value = obj_PR2
                    self._solve_time = time.time() - self._start_time
                    return LpStatusOptimal
            else:
                solution_PR2, obj_PR2 = None, float('inf')
            
            # Step 2: Generate solution pools using depot insertions
            pool_PR1 = self._generate_solution_pool(route_PR1, obj_PR1) if route_PR1 else []
            pool_PR2 = self._generate_solution_pool(route_PR2, obj_PR2) if route_PR2 else []
            
            # Step 3: Local search on both pools
            best_solution = None
            best_objective = float('inf')
            
            # Use both pools
            for pool_idx, pool in enumerate([pool_PR1, pool_PR2]):
                if pool:  # Only process non-empty pools
                    for _ in range(2):  # κrs/2 times per pool as per paper
                        local_solution = self._local_search(pool.copy())
                        if local_solution:
                            route, assignments, obj = local_solution
                            if obj < best_objective:
                                best_objective = obj
                                best_solution = (route, assignments, obj)
            
            if best_solution:
                self.final_route, self.assignments, self.objective_value = best_solution
                self._solve_time = time.time() - self._start_time
                return LpStatusOptimal
            
            return LpStatusNotSolved
            
        except Exception as e:
            print(f"\nERROR: Solver failed with error: {str(e)}")
            return LpStatusNotSolved
    
    def _generate_route_PR1(self) -> Optional[List[str]]:
        """PR1: Move to position with most satisfiable customers"""
        try:
            route = [self.gamma]
            processed_customers = set()
            current_pos = self.gamma
            current_time = 0
            
            iteration = 0
            while len(processed_customers) < len(self.instance.C) and iteration < 100:  # Added iteration limit
                iteration += 1
                
                # Get customers satisfiable from current position
                satisfiable = {
                    k for k in (self.instance.C - processed_customers)
                    if current_time + self.instance.robot_travel_times.get((current_pos, k), float('inf')) <= self.instance.deadlines[k]
                }
                processed_customers.update(satisfiable)
                
                # Find location with most satisfiable unprocessed customers
                best_location = None
                max_satisfiable = -1
                
                remaining_locations = (self.instance.D.union(self.instance.R)) - set(route)
                
                if not remaining_locations:
                    if processed_customers == self.instance.C:
                        return route
                    else:
                        return None
                
                for loc in remaining_locations:
                    next_time = current_time + self._get_travel_time(current_pos, loc)
                    count = sum(1 for k in (self.instance.C - processed_customers)
                              if next_time + self.instance.robot_travel_times.get((loc, k), float('inf')) <= self.instance.deadlines[k])
                    if count > max_satisfiable:
                        max_satisfiable = count
                        best_location = loc
                
                if best_location is None:
                    unvisited_depots = self.instance.R - set(route)
                    if unvisited_depots:
                        best_location = min(unvisited_depots,
                                          key=lambda r: self._get_travel_time(current_pos, r))
                    else:
                        return None
                
                route.append(best_location)
                current_pos = best_location
                current_time += self._get_travel_time(route[-2], route[-1])
                
                if len(route) > len(self.instance.C) + len(self.instance.D) + len(self.instance.R):
                    return None
            
            return route
            
        except Exception as e:
            print(f"\nError in generate_route_PR1: {str(e)}")
            return None

    def _generate_route_PR2(self) -> Optional[List[str]]:
        """PR2: Move to position with highest urgency"""
        try:
            route = [self.gamma]
            processed_customers = set()
            current_pos = self.gamma
            current_time = 0
            
            # Assign each customer to nearest depot or drop-off point
            nearest_point = {}
            for k in self.instance.C:
                options = self.instance.D.union(self.instance.R)
                if options:
                    nearest = min(options,
                                key=lambda p: self.instance.robot_travel_times.get((p, k), float('inf')))
                    nearest_point[k] = nearest
                else:
                    return None
            
            iteration = 0
            while len(processed_customers) < len(self.instance.C) and iteration < 100:  # Added iteration limit
                iteration += 1
                
                best_location = None
                min_urgency = float('inf')
                
                remaining_locations = (self.instance.D.union(self.instance.R)) - set(route)
                
                if not remaining_locations:
                    if processed_customers == self.instance.C:
                        return route
                    else:
                        return None
                
                for loc in remaining_locations:
                    next_time = current_time + self._get_travel_time(current_pos, loc)
                    assigned_customers = {k for k, p in nearest_point.items() 
                                       if p == loc and k not in processed_customers}
                    
                    if assigned_customers:
                        min_budget = min(self.instance.deadlines[k] - 
                                       (next_time + self.instance.robot_travel_times.get((loc, k), float('inf')))
                                       for k in assigned_customers)
                        if min_budget < min_urgency:
                            min_urgency = min_budget
                            best_location = loc
                
                if best_location is None:
                    unvisited_depots = self.instance.R - set(route)
                    if unvisited_depots:
                        best_location = min(unvisited_depots,
                                          key=lambda r: self._get_travel_time(current_pos, r))
                    else:
                        return None
                
                route.append(best_location)
                current_pos = best_location
                current_time += self._get_travel_time(route[-2], route[-1])
                
                # Update processed customers
                for k in set(self.instance.C) - processed_customers:
                    if nearest_point.get(k) == best_location:
                        if current_time + self.instance.robot_travel_times.get((best_location, k), float('inf')) <= self.instance.deadlines[k]:
                            processed_customers.add(k)
                
                if len(route) > len(self.instance.C) + len(self.instance.D) + len(self.instance.R):
                    return None
            
            return route
            
        except Exception as e:
            print(f"\nError in generate_route_PR2: {str(e)}")
            return None
    
    def _generate_solution_pool(self, initial_route: Optional[List[str]], initial_obj: float) -> List[Tuple[List[str], float]]:
        """Generate pool of solutions using depot insertions"""
        if initial_route is None or initial_obj == float('inf'):
            return []
            
        pool = [(initial_route, initial_obj)]
        
        try:
            # Find consecutive drop-off points in route
            for i in range(len(initial_route) - 1):
                if (initial_route[i] in self.instance.D and 
                    initial_route[i+1] in self.instance.D):
                    
                    # Try inserting each depot between them
                    for depot in self.instance.R:
                        if depot not in initial_route[i:i+2]:
                            new_route = initial_route[:i+1] + [depot] + initial_route[i+1:]
                            assignments, obj = self.heuristic_solver.solve_route(new_route)
                            
                            if assignments is not None and obj != float('inf') and obj < initial_obj:
                                pool.append((new_route, obj))
            
            return pool
        except Exception as e:
            print(f"\nError in generate_solution_pool: {str(e)}")
            return [(initial_route, initial_obj)]

    def _local_search(self, pool: List[Tuple[List[str], float]], 
                     iterations_ls1: int = 100,
                     iterations_ls2: int = 200) -> Optional[Tuple[List[str], Dict[str, str], float]]:
        """Local search procedure"""
        if not pool:
            return None
    
        try:
            # Filter out any invalid solutions from pool
            valid_pool = [(route, obj) for route, obj in pool if route and obj != float('inf')]
            if not valid_pool:
                return None
                
            current_solution = min(valid_pool, key=lambda x: x[1])  # Get solution with lowest objective
            current_route, current_obj = current_solution
    
            if current_route is None or current_obj == float('inf'):
                return None

            # Early termination if optimal
            if current_obj == 0:
                current_assignments, _ = self.heuristic_solver.solve_route(current_route)
                return current_route, current_assignments, current_obj
                
            current_assignments, _ = self.heuristic_solver.solve_route(current_route)
            if current_assignments is None:
                return None
            
            # First phase with adaptive iterations
            no_improvement_count = 0
            for iter in range(iterations_ls1):
                if no_improvement_count >= 20:  # Stop if no improvement for 20 iterations
                    break
                
                neighborhood = random.choice(['N1', 'N2', 'N3', 'N4'])
                new_route = self._apply_neighborhood(current_route, neighborhood)
            
                if new_route:
                    assignments, obj = self.heuristic_solver.solve_route(new_route)
                    if assignments is not None and obj != float('inf'):
                        if obj == 0:  # Found optimal solution
                            return new_route, assignments, obj
                        if obj < current_obj:
                            current_route = new_route
                            current_obj = obj
                            current_assignments = assignments
                            valid_pool = [(r, o) for r, o in valid_pool if o < current_obj]
                            no_improvement_count = 0
                        elif obj >= current_obj * 1.5:  # If solution is much worse, increment counter more
                            no_improvement_count += 2
                        else:
                            no_improvement_count += 1
        
            # Second phase with adaptive iterations
            no_improvement_count = 0
            for iter in range(iterations_ls2):
                if no_improvement_count >= 20:
                    break
                
                neighborhood = random.choice(['N1', 'N2', 'N3', 'N4'])
                new_route = self._apply_neighborhood(current_route, neighborhood)
            
                if new_route:
                    assignments, obj = self.heuristic_solver.solve_route(new_route)
                    if assignments is not None and obj != float('inf'):
                        if obj == 0:  # Found optimal solution
                            return new_route, assignments, obj
                        if obj < current_obj:
                            current_route = new_route
                            current_obj = obj
                            current_assignments = assignments
                            no_improvement_count = 0
                        elif obj >= current_obj * 1.5:  # If solution is much worse, increment counter more
                            no_improvement_count += 2
                        else:
                            no_improvement_count += 1
        
            return current_route, current_assignments, current_obj
            
        except Exception as e:
            print(f"\nError in local_search: {str(e)}")
            return None
    
    def _apply_neighborhood(self, route: List[str], neighborhood: str) -> Optional[List[str]]:
        """Apply neighborhood operation to route"""
        # Early pruning checks
        if not route or len(route) < 2:
            return None
        
        if len(route) > len(self.instance.C) + len(self.instance.D) + len(self.instance.R):
            return None
        
        # Check if route contains required minimum elements
        if not any(loc in self.instance.R for loc in route):  # Must have at least one depot
            return None
        
        if not any(loc in self.instance.D for loc in route):  # Must have at least one drop-off
            return None

        try:
            if neighborhood == 'N1':
                # Remove random position (not single depot)
                depot_counts = sum(1 for loc in route if loc in self.instance.R)
                if depot_counts > 1:
                    # Only try to remove non-critical positions
                    removable_indices = [i for i in range(1, len(route)) 
                                       if not (route[i] in self.instance.R and depot_counts <= 1)]
                    if removable_indices:
                        idx = random.choice(removable_indices)
                        new_route = route[:idx] + route[idx+1:]
                        if len(new_route) >= 2:  # Ensure route still valid after removal
                            return new_route
                    
            elif neighborhood == 'N2':
                # Insert random depot/drop-off, but be smarter about insertion
                remaining_locations = self.instance.D.union(self.instance.R) - set(route)
                if remaining_locations:  # Only insert if we have unused locations
                    location = random.choice(list(remaining_locations))
                    # Don't insert between consecutive drop-offs if it's a depot
                    valid_positions = [i for i in range(1, len(route)) 
                                     if not (location in self.instance.R and 
                                           i > 0 and i < len(route) and
                                           route[i-1] in self.instance.D and 
                                           route[i] in self.instance.D)]
                    if valid_positions:
                        idx = random.choice(valid_positions)
                        return route[:idx] + [location] + route[idx:]
            
            elif neighborhood == 'N3':
                # Swap two positions, but be smarter about which positions to swap
                valid_indices = range(1, len(route))
                if len(valid_indices) >= 2:
                    idx1, idx2 = random.sample(list(valid_indices), 2)
                    # Don't swap if it would create invalid depot sequence
                    if not (route[idx1] in self.instance.R and route[idx2] in self.instance.R):
                        new_route = route.copy()
                        new_route[idx1], new_route[idx2] = new_route[idx2], new_route[idx1]
                        return new_route
                
            elif neighborhood == 'N4':
                # Insert closest depot before drop-off point
                drop_positions = [i for i, loc in enumerate(route) if loc in self.instance.D]
                unvisited_depots = self.instance.R - set(route)
                if drop_positions and unvisited_depots:
                    pos = random.choice(drop_positions)
                    # Find depot that minimizes total travel time
                    nearest_depot = min(unvisited_depots,
                                     key=lambda r: (self._get_travel_time(route[pos-1], r) +
                                                  self._get_travel_time(r, route[pos])))
                    return route[:pos] + [nearest_depot] + route[pos:]
        
            return None
        
        except Exception as e:
            print(f"\nError in apply_neighborhood: {str(e)}")
            return None
    
    def get_solution(self) -> Optional[Dict]:
        """Return the solution details"""
        if not hasattr(self, 'objective_value') or self.objective_value is None:
            return None
            
        try:
            solution = {
                'objective_value': float(self.objective_value),
                'route': self.final_route,
                'customer_assignments': self.assignments,
                'coordinates': self.instance.coordinates,
                'completion_time': getattr(self, '_solve_time', None),
                'is_optimal': False  # Since we're using heuristic
            }
            return solution
            
        except Exception as e:
            print(f"\nError extracting solution: {str(e)}")
            return None
    
def solve_single_instance(instance_params: Dict, seed: int, 
                         time_limit: Optional[int] = None,
                         gap: Optional[float] = None) -> Optional[Dict]:
    """Solve a single instance for parallel processing"""
    def solve_with_timeout():
        try:
            generator = TBRDInstanceGenerator(**instance_params)
            instance = generator.generate_instance(seed=seed)
            model = TBRDModel(instance)
            status = model.solve(time_limit=time_limit, gap=gap)
            
            solution = model.get_solution()
            if solution:
                solution['instance_id'] = seed
                solution['status'] = status
                return solution
            return None
        except Exception as e:
            print(f"\nError solving instance {seed}: {str(e)}")
            return None

    if time_limit:
        with ThreadPoolExecutor(max_workers=1) as executor:
            future = executor.submit(solve_with_timeout)
            try:
                result = future.result(timeout=time_limit)
                return result
            except FutureTimeoutError:
                print(f"\nInstance {seed} exceeded time limit of {time_limit} seconds")
                return None
    else:
        return solve_with_timeout()
        
def solve_chunk(params, instance_ids, t_limit, g):
    chunk_results = []
    for i in instance_ids:
        try:
            result = solve_single_instance(params, i, t_limit, g)
            if result is not None:
                chunk_results.append(result)
        except Exception as e:
            print(f"\nError solving instance {i}: {str(e)}")
    return chunk_results

def run_experiment(instance_params: Dict, 
                  num_instances: int = 1,
                  time_limit: Optional[int] = None,
                  gap: Optional[float] = None) -> List[Dict]:
    """
    Run experiments with specified parameters using parallel processing
    """
    # Determine number of cores to use (leave one core free)
    num_processes = max(1, multiprocessing.cpu_count() - 1)
    
    # Create chunks of instances for parallel processing
    chunks = [(instance_params, list(range(i, min(i + num_processes, num_instances))), time_limit, gap) 
             for i in range(0, num_instances, num_processes)]
    
    # Process chunks in parallel
    results = []
    with multiprocessing.Pool(num_processes) as pool:
        chunk_results = pool.starmap(solve_chunk, chunks)
        for chunk in chunk_results:
            results.extend(chunk)
    
    # Calculate statistics
    if results:
        best_objective = float('inf')
        best_solutions_count = 0
        optima_found = 0
        best_solutions_total_time = 0
        
        for solution in results:
            obj_val = float(solution['objective_value'])
            
            if obj_val == 0:
                optima_found += 1
            
            if obj_val < best_objective:
                best_objective = obj_val
                best_solutions_count = 1
                best_solutions_total_time = solution['completion_time']
            elif obj_val == best_objective:
                best_solutions_count += 1
                best_solutions_total_time += solution['completion_time']
        
        print("\n" + "="*50)
        print("FINAL SUMMARY STATISTICS")
        print("="*50)
        print(f"Total instances solved: {len(results)}")
        print(f"\nBest Solution Statistics:")
        print(f"Best objective value found: {best_objective}")
        print(f"Number of times best solution was found: {best_solutions_count}")
        print(f"Number of optima found (obj = 0): {optima_found}")
        if best_solutions_count > 0:
            print(f"Average CPU time for best solutions: {best_solutions_total_time/best_solutions_count:.2f} seconds")
        
        print(f"\nOverall Statistics:")
        print(f"Average objective value: {sum(float(r['objective_value']) for r in results) / len(results):.2f}")
        print(f"Average solution time: {sum(r['completion_time'] for r in results) / len(results):.2f} seconds")
        print(f"Using {num_processes} parallel processes")
    else:
        print("\nNo solutions found.")
    
    return results        
        
if __name__ == "__main__":
    # Medium-sized instance with more relaxed parameters
    instance_params = {
        'w': 2,                # 5 km square area (increased from 2)
        'num_customers': 6,   # 10 customers
        'num_dropoff': 6,      # 5 drop-off points
        'num_depots': 4,       # 3 robot depots
        'truck_speed': 30,     # 40 km/h (increased from 30)
        'robot_speed': 5,     # 20 km/h (increased from 15)
        'truck_capacity': 2,   # Maximum 5 robots (increased from 3)
        'rho_min': 2.0,        # Minimum deadline factor (increased from 1.2)
        'rho_max': 4.0,        # Maximum deadline factor (increased from 2.0)
        'w_min': 1,           # Minimum customer weight
        'w_max': 1            # Maximum customer weight
    }
    
    # Run experiment with two instances
    results = run_experiment(
        instance_params,
        num_instances=10,      # Two instances
        time_limit=200,       # 5 minutes per instance
        gap=0.1              # 10% gap tolerance
    )